MENYIAPKAN ENVIRONMENT TUGAS AKHIR

In [1]:
# -*- coding: utf-8 -*-
import sys
import getopt
import csv
from datetime import datetime
from GetOldTweets import got3 as got
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

MENGUMPULKAN TWEETS DENGAN QUERY TERTENTU

In [2]:
#mendapatkan query yang diinginkan
def getTweet(argv):
    if len(argv) == 0:
        print('You must pass some parameters. Use \"-h\" to help.')
        return

    if len(argv) == 1 and argv[0] == '-h':
        f = open('exporter_help_text.txt', 'r')
        print(f.read())
        f.close()

        return

    try:
        opts, args = getopt.getopt(argv, "", (
        "username=", "near=", "within=", "since=", "until=", "querysearch=", "toptweets", "maxtweets=", "output=", "lang="))

        tweetCriteria = got.manager.TweetCriteria()
        outputFileName = "TweetsTA.csv"

        for opt, arg in opts:
            if opt == '--username':
                tweetCriteria.username = arg

            elif opt == '--since':
                tweetCriteria.since = arg

            elif opt == '--until':
                tweetCriteria.until = arg

            elif opt == '--querysearch':
                tweetCriteria.querySearch = arg

            elif opt == '--toptweets':
                tweetCriteria.topTweets = True

            elif opt == '--maxtweets':
                tweetCriteria.maxTweets = int(arg)

            elif opt == '--near':
                tweetCriteria.near = '"' + arg + '"'

            elif opt == '--within':
                tweetCriteria.within = '"' + arg + '"'

            elif opt == '--output':
                outputFileName = arg

            elif opt == '--lang':
                tweetCriteria.lang = arg

        #membuat file csv
        outputFile = csv.writer(open(outputFileName, "w", encoding='utf-8-sig', newline=''), delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)

        outputFile.writerow(
            ['username','date','retweets','favorites','text','geo','mentions','hashtags','id','permalink', 'emoji'])

        print('Collecting tweets...\n')

        def receiveBuffer(tweets):
            for t in tweets:
                add_list = []
                if (isinstance(t.emojis, list)):
                    emoji = ' '.join(t.emojis)
                else:
                    emoji = t.emojis
                for each in [t.username, t.date.strftime("%Y-%m-%d %H:%M"), t.retweets, t.favorites, t.text, t.geo, t.mentions, t.hashtags, t.id, t.permalink, emoji]:
                    add_list.append(each)
                #mendapatkan tweet sesuai query dan memasukkan ke csv
                outputFile.writerow(add_list)
            print('%d tweets saved on file...\n' % len(tweets))

        got.manager.TweetManager.getTweets(tweetCriteria, receiveBuffer)

    except :
        print('Arguments parse error, failed collecting tweets')
    finally:
        print('Done. Output file generated "%s".' % outputFileName)



In [5]:
query = [,"--near","Indonesia","--within","15mi","--since","2019-06-01","--until","2019-07-02"]
getTweet(query)


Twitter weird response. Try to see on browser: https://twitter.com/search?q=%20since%3A2019-06-01%20until%3A2019-07-02&src=typd
Unexpected error: <class 'urllib.error.HTTPError'>
Arguments parse error, failed collecting tweets
Done. Output file generated "TweetsTA.csv".


In [2]:
#load dataset
tweets = pd.read_csv('TweetsTA.csv')
# tweets.head(1)
# tweets[['username','date','text','geo']].head(10)

FILTRASI TWEETS DENGAN NER
https://yudiwbs.wordpress.com/2018/03/29/ner-named-entity-recognition-dengan-anago-python-keras/

In [3]:
#mempersiapkan environment anago 
import anago
from anago.reader import load_data_and_labels
import os
import numpy as np
import random as rn
import gensim.models.keyedvectors as W2V

Using TensorFlow backend.
D:\ITS\Tugas Akhir\TA\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
#mengakses file yang trainer, validator, dan tester
namaFileTrain = "train.txt"
namaFileValid = "valid.txt"
namaFileTest = "test.txt"
x_train, y_train = load_data_and_labels(namaFileTrain)
x_valid, y_valid = load_data_and_labels(namaFileValid)
x_test, y_test = load_data_and_labels(namaFileTest)

In [5]:
# karena hasil tdk konsisten, random seednya diisi manual
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)

# mengatur parameter
model = anago.Sequence(char_emb_size=25, word_emb_size=100, char_lstm_units=25,
                       word_lstm_units=50, dropout=0.5, char_feature=True, crf=True,
                       batch_size=20, optimizer='adam', learning_rate=0.001, lr_decay=0.9,
                       clip_gradients=5.0, max_epoch=3, early_stopping=True, patience=3, train_embeddings=True,
                       max_checkpoints_to_keep=5, log_dir=None)
model.train(x_train, y_train, x_valid, y_valid)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
44/44 [==============================] - 11s 259ms/step - loss: 119.51120s - loss: 119.7
 - f1: 0.00
Epoch 2/3
44/44 [==============================] - 5s 121ms/step - loss: 104.2277
 - f1: 0.00
Epoch 3/3
44/44 [==============================] - 5s 121ms/step - loss: 101.7719
 - f1: 0.93


In [6]:
print("\n\nEvaluasi Test:")
model.eval(x_test, y_test)
 
words = 'Budi Martami kuliah di UPI yang berlokasi di Bandung'.split()
print(model.analyze(words))
 
words = 'PDIP yang dikawal Megawati menang dalam Pilkada DKI Jakarta'.split()
print(model.analyze(words))



Evaluasi Test:
 - f1: 1.36
{'words': ['Budi', 'Martami', 'kuliah', 'di', 'UPI', 'yang', 'berlokasi', 'di', 'Bandung'], 'entities': []}
{'words': ['PDIP', 'yang', 'dikawal', 'Megawati', 'menang', 'dalam', 'Pilkada', 'DKI', 'Jakarta'], 'entities': []}


In [7]:
#menyiapkan environment untuk preprocessing tweet
import csv
import pandas as pd
import numpy as np
import html
import re
import json
import emoji
# import spacy
# from spacy.lang.id import Indonesian

In [9]:
#memuat tweet yang dikumpulkan sebelumnya (load dataset)
tweets = pd.read_csv('TweetsTA.csv')
# tweets.head(5)

In [10]:
#Preprocessing Tweets
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# nlp = Indonesian()
# pd.set_option('max_colwidth', 100)
# stopwords = spacy.lang.id.stop_words.STOP_WORDS

# load sastrawi's Bahasa Indonesia stopwords as variable called 'stopwords'
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
extender = ["ada", "adalah", "adanya", "adapun", "agak", "agaknya", "agar", "akan", "akankah", "akhir", "akhiri", "akhirnya", "aku", "akulah", "amat", "amatlah", "anda", "andalah", "antar", "antara", "antaranya", "apa", "apaan", "apabila", "apakah", "apalagi", "apatah", "artinya", "asal", "asalkan", "atas", "atau", "ataukah", "ataupun", "awal", "awalnya", "bagai", "bagaikan", "bagaimana", "bagaimanakah", "bagaimanapun", "bagi", "bagian", "bahkan", "bahwa", "bahwasanya", "baik", "bakal", "bakalan", "balik", "banyak", "bapak", "baru", "bawah", "beberapa", "begini", "beginian", "beginikah", "beginilah", "begitu", "begitukah", "begitulah", "begitupun", "bekerja", "belakang", "belakangan", "belum", "belumlah", "benar", "benarkah", "benarlah", "berada", "berakhir", "berakhirlah", "berakhirnya", "berapa", "berapakah", "berapalah", "berapapun", "berarti", "berawal", "berbagai", "berdatangan", "beri", "berikan", "berikut", "berikutnya", "berjumlah", "berkali-kali", "berkata", "berkehendak", "berkeinginan", "berkenaan", "berlainan", "berlalu", "berlangsung", "berlebihan", "bermacam", "bermacam-macam", "bermaksud", "bermula", "bersama", "bersama-sama", "bersiap", "bersiap-siap", "bertanya", "bertanya-tanya", "berturut", "berturut-turut", "bertutur", "berujar", "berupa", "besar", "betul", "betulkah", "biasa", "biasanya", "bila", "bilakah", "bisa", "bisakah", "boleh", "bolehkah", "bolehlah", "buat", "bukan", "bukankah", "bukanlah", "bukannya", "bulan", "bung", "cara", "caranya", "cukup", "cukupkah", "cukuplah", "cuma", "dahulu", "dalam", "dan", "dapat", "dari", "daripada", "datang", "dekat", "demi", "demikian", "demikianlah", "dengan", "depan", "di", "dia", "diakhiri", "diakhirinya", "dialah", "diantara", "diantaranya", "diberi", "diberikan", "diberikannya", "dibuat", "dibuatnya", "didapat", "didatangkan", "digunakan", "diibaratkan", "diibaratkannya", "diingat", "diingatkan", "diinginkan", "dijawab", "dijelaskan", "dijelaskannya", "dikarenakan", "dikatakan", "dikatakannya", "dikerjakan", "diketahui", "diketahuinya", "dikira", "dilakukan", "dilalui", "dilihat", "dimaksud", "dimaksudkan", "dimaksudkannya", "dimaksudnya", "diminta", "dimintai", "dimisalkan", "dimulai", "dimulailah", "dimulainya", "dimungkinkan", "dini", "dipastikan", "diperbuat", "diperbuatnya", "dipergunakan", "diperkirakan", "diperlihatkan", "diperlukan", "diperlukannya", "dipersoalkan", "dipertanyakan", "dipunyai", "diri", "dirinya", "disampaikan", "disebut", "disebutkan", "disebutkannya", "disini", "disinilah", "ditambahkan", "ditandaskan", "ditanya", "ditanyai", "ditanyakan", "ditegaskan", "ditujukan", "ditunjuk", "ditunjuki", "ditunjukkan", "ditunjukkannya", "ditunjuknya", "dituturkan", "dituturkannya", "diucapkan", "diucapkannya", "diungkapkan", "dong", "dua", "dulu", "empat", "enggak", "enggaknya", "entah", "entahlah", "guna", "gunakan", "hal", "hampir", "hanya", "hanyalah", "hari", "harus", "haruslah", "harusnya", "hendak", "hendaklah", "hendaknya", "hingga", "ia", "ialah", "ibarat", "ibaratkan", "ibaratnya", "ibu", "ikut", "ingat", "ingat-ingat", "ingin", "inginkah", "inginkan", "ini", "inikah", "inilah", "itu", "itukah", "itulah", "jadi", "jadilah", "jadinya", "jangan", "jangankan", "janganlah", "jauh", "jawab", "jawaban", "jawabnya", "jelas", "jelaskan", "jelaslah", "jelasnya", "jika", "jikalau", "juga", "jumlah", "jumlahnya", "justru", "kala", "kalau", "kalaulah", "kalaupun", "kalian", "kami", "kamilah", "kamu", "kamulah", "kan", "kapan", "kapankah", "kapanpun", "karena", "karenanya", "kasus", "kata", "katakan", "katakanlah", "katanya", "ke", "keadaan", "kebetulan", "kecil", "kedua", "keduanya", "keinginan", "kelamaan", "kelihatan", "kelihatannya", "kelima", "keluar", "kembali", "kemudian", "kemungkinan", "kemungkinannya", "kenapa", "kepada", "kepadanya", "kesampaian", "keseluruhan", "keseluruhannya", "keterlaluan", "ketika", "khususnya", "kini", "kinilah", "kira", "kira-kira", "kiranya", "kita", "kitalah", "kok", "kurang", "lagi", "lagian", "lah", "lain", "lainnya", "lalu", "lama", "lamanya", "lanjut", "lanjutnya", "lebih", "lewat", "lima", "luar", "macam", "maka", "makanya", "makin", "malah", "malahan", "mampu", "mampukah", "mana", "manakala", "manalagi", "masa", "masalah", "masalahnya", "masih", "masihkah", "masing", "masing-masing", "mau", "maupun", "melainkan", "melakukan", "melalui", "melihat", "melihatnya", "memang", "memastikan", "memberi", "memberikan", "membuat", "memerlukan", "memihak", "meminta", "memintakan", "memisalkan", "memperbuat", "mempergunakan", "memperkirakan", "memperlihatkan", "mempersiapkan", "mempersoalkan", "mempertanyakan", "mempunyai", "memulai", "memungkinkan", "menaiki", "menambahkan", "menandaskan", "menanti", "menanti-nanti", "menantikan", "menanya", "menanyai", "menanyakan", "mendapat", "mendapatkan", "mendatang", "mendatangi", "mendatangkan", "menegaskan", "mengakhiri", "mengapa", "mengatakan", "mengatakannya", "mengenai", "mengerjakan", "mengetahui", "menggunakan", "menghendaki", "mengibaratkan", "mengibaratkannya", "mengingat", "mengingatkan", "menginginkan", "mengira", "mengucapkan", "mengucapkannya", "mengungkapkan", "menjadi", "menjawab", "menjelaskan", "menuju", "menunjuk", "menunjuki", "menunjukkan", "menunjuknya", "menurut", "menuturkan", "menyampaikan", "menyangkut", "menyatakan", "menyebutkan", "menyeluruh", "menyiapkan", "merasa", "mereka", "merekalah", "merupakan", "meski", "meskipun", "meyakini", "meyakinkan", "minta", "mirip", "misal", "misalkan", "misalnya", "mula", "mulai", "mulailah", "mulanya", "mungkin", "mungkinkah", "nah", "naik", "namun", "nanti", "nantinya", "nyaris", "nyatanya", "oleh", "olehnya", "pada", "padahal", "padanya", "pak", "paling", "panjang", "pantas", "para", "pasti", "pastilah", "penting", "pentingnya", "per", "percuma", "perlu", "perlukah", "perlunya", "pernah", "persoalan", "pertama", "pertama-tama", "pertanyaan", "pertanyakan", "pihak", "pihaknya", "pukul", "pula", "pun", "punya", "rasa", "rasanya", "rata", "rupanya", "saat", "saatnya", "saja", "sajalah", "saling", "sama", "sama-sama", "sambil", "sampai", "sampai-sampai", "sampaikan", "sana", "sangat", "sangatlah", "satu", "saya", "sayalah", "se", "sebab", "sebabnya", "sebagai", "sebagaimana", "sebagainya", "sebagian", "sebaik", "sebaik-baiknya", "sebaiknya", "sebaliknya", "sebanyak", "sebegini", "sebegitu", "sebelum", "sebelumnya", "sebenarnya", "seberapa", "sebesar", "sebetulnya", "sebisanya", "sebuah", "sebut", "sebutlah", "sebutnya", "secara", "secukupnya", "sedang", "sedangkan", "sedemikian", "sedikit", "sedikitnya", "seenaknya", "segala", "segalanya", "segera", "seharusnya", "sehingga", "seingat", "sejak", "sejauh", "sejenak", "sejumlah", "sekadar", "sekadarnya", "sekali", "sekali-kali", "sekalian", "sekaligus", "sekalipun", "sekarang", "sekarang", "sekecil", "seketika", "sekiranya", "sekitar", "sekitarnya", "sekurang-kurangnya", "sekurangnya", "sela", "selain", "selaku", "selalu", "selama", "selama-lamanya", "selamanya", "selanjutnya", "seluruh", "seluruhnya", "semacam", "semakin", "semampu", "semampunya", "semasa", "semasih", "semata", "semata-mata", "semaunya", "sementara", "semisal", "semisalnya", "sempat", "semua", "semuanya", "semula", "sendiri", "sendirian", "sendirinya", "seolah", "seolah-olah", "seorang", "sepanjang", "sepantasnya", "sepantasnyalah", "seperlunya", "seperti", "sepertinya", "sepihak", "sering", "seringnya", "serta", "serupa", "sesaat", "sesama", "sesampai", "sesegera", "sesekali", "seseorang", "sesuatu", "sesuatunya", "sesudah", "sesudahnya", "setelah", "setempat", "setengah", "seterusnya", "setiap", "setiba", "setibanya", "setidak-tidaknya", "setidaknya", "setinggi", "seusai", "sewaktu", "siap", "siapa", "siapakah", "siapapun", "sini", "sinilah", "soal", "soalnya", "suatu", "sudah", "sudahkah", "sudahlah", "supaya", "tadi", "tadinya", "tahu", "tahun", "tak", "tambah", "tambahnya", "tampak", "tampaknya", "tandas", "tandasnya", "tanpa", "tanya", "tanyakan", "tanyanya", "tapi", "tegas", "tegasnya", "telah", "tempat", "tengah", "tentang", "tentu", "tentulah", "tentunya", "tepat", "terakhir", "terasa", "terbanyak", "terdahulu", "terdapat", "terdiri", "terhadap", "terhadapnya", "teringat", "teringat-ingat", "terjadi", "terjadilah", "terjadinya", "terkira", "terlalu", "terlebih", "terlihat", "termasuk", "ternyata", "tersampaikan", "tersebut", "tersebutlah", "tertentu", "tertuju", "terus", "terutama", "tetap", "tetapi", "tiap", "tiba", "tiba-tiba", "tidak", "tidakkah", "tidaklah", "tiga", "tinggi", "toh", "tunjuk", "turut", "tutur", "tuturnya", "ucap", "ucapnya", "ujar", "ujarnya", "umum", "umumnya", "ungkap", "ungkapnya", "untuk", "usah", "usai", "waduh", "wah", "wahai", "waktu", "waktunya", "walau", "walaupun", "wong", "yaitu", "yakin", "yakni", "yang"]
stopwords.extend(extender)
# print(stopwords)
def normalize_repeated_char(tokenized_sentence):
    alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
                'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
                'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    tokenized_sentence = ' '.join(tokenized_sentence)
    for i in range(len(alphabet)):
        charac_long = 5
        while charac_long >= 2:
            char = alphabet[i] * charac_long
            tokenized_sentence = tokenized_sentence.replace(char, alphabet[i])
            charac_long -= 1        
    tokenized_sentence = tokenized_sentence.split()    
    return tokenized_sentence

def delete_emoji(sentence):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    for word in sentence:
        for letter in word:
            print(letter)
#             letter = emoji_pattern.sub(r'', letter)
#     deleted_emoji = emoji_pattern.sub(r'', ContentNoSlang)
#     return deleted_emoji

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

def preprocessing_tweets_first(text):
    clean = html.unescape(text) #Handling Unicode HTML
    clean = re.sub("[0-9]", "", clean) #RemoveNumber
    clean = re.sub(r"http\S+", "", clean) #Remove links
    clean = deEmojify(clean) #Remove Emoji
    clean = " ".join(re.findall("[#a-zA-Z]{3,}", clean)) #Remove Puntc
    content = list()
    content.append(clean)
    return " ".join(content)
 

In [11]:
tweets['clean'] = tweets['text'].apply(preprocessing_tweets_first)
tweets = tweets.drop(["retweets", "favorites", "mentions", "hashtags", "id", "permalink", "emoji"], axis=1)

In [12]:
tweets.head(10)

,username,date,text,geo,clean
0,PenguntaiKata,2019-07-02 06:59,Jadi pindah negara gak ? Jokowi menang tuh 😂😂😂😂,NaN,Jadi pindah negara gak Jokowi menang tuh
1,SutrisnoAhmadd,2019-07-02 06:59,Bilg curg tpi ga bisa buktikan itu fitnah..tau...,NaN,Bilg curg tpi bisa buktikan itu fitnah tau kan...
2,RencanaTrading,2019-07-02 06:59,Thanks Jul. Baru sadar kalau ada berita itu. ...,NaN,Thanks Jul Baru sadar kalau ada berita itu Ter...
3,anwar_suprijadi,2019-07-02 06:59,Mestinya dulu segera diruwat.. Trus ganti nam...,NaN,Mestinya dulu segera diruwat Trus ganti nama j...
4,TlatahBocah,2019-07-02 06:59,5 - 7/7/19 Festival Lima Gunung XVIII | Padepo...,NaN,Festival Lima Gunung XVIII Padepokan Seni Tjip...
5,doenklanus88,2019-07-02 06:59,"Ya lebih baik hadir,itu memberikan contoh jiwa...",NaN,lebih baik hadir itu memberikan contoh jiwa ke...
6,GATRA_News,2019-07-02 06:59,"Teken Perpres 37/2019, Jokowi Dinilai Buta Per...",NaN,Teken Perpres Jokowi Dinilai Buta Pertahanan P...
7,srisumartie,2019-07-02 06:59,"Jangan remehkan ""Nafsunya para ABJ"" 😁😁😁 mpe se...",NaN,Jangan remehkan Nafsunya para ABJ mpe segituny...
8,veyedov,2019-07-02 06:58,"Lu dirampok dijarah, anak2 lu dipukulin trus l...",NaN,dirampok dijarah anak dipukulin trus bilang se...
9,AfanHakimi,2019-07-02 06:58,"Mantap Pak Jokowi..Abaikan saja para kampret, ...",NaN,Mantap Pak Jokowi Abaikan saja para kampret sa...


EKSTRAKSI TRENDING ISSUE DARI TWEETS

Filtrasi Tweets

In [13]:
tweets.head(1)

,username,date,text,geo,clean
0,PenguntaiKata,2019-07-02 06:59,Jadi pindah negara gak ? Jokowi menang tuh 😂😂😂😂,NaN,Jadi pindah negara gak Jokowi menang tuh


In [124]:
# tweet = tweets['clean'].tolist()
# i=0
# for a in tweet:
#     if len(a.split()) > 2:
#         model.analyze(a.split())
#         print(i)
#         i+=1

In [14]:
#filtrasi tweets dengan menggunakan NER
index = 0
tweets_with_entity = pd.DataFrame(columns=['text', 'index'])
for sent in tweets['clean'].tolist():
    if len(sent.split()) >= 3:
        result = model.analyze(sent.split())
        if result.get("entities") != []:
            tweets_with_entity = tweets_with_entity.append({'text':sent, 'index': index}, ignore_index=True)
#             print(sent)
#     print(index)
#     index+=1

In [15]:
tweets_with_entity.head(1)

,text,index
0,budaya barat tuh,0


#CSV File
with open('clean.csv', 'w', encoding="utf-8") as csv_tweet:
    fieldnames_tweet = ['id','text']
    writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
    writer_tweet.writeheader()
    id = 0
    for index, tweet in tweets.iterrows():
        words = tweet["clean"].split()
        if (len(words)) > 2:
            writer_tweet.writerow({
                'id': index, 
                'text': tweet["clean"]
            })

In [16]:
# Preprocessing tweets tahap2
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stopword(words):
    clean = []
    for word in words:
        if word not in stopwords:
            clean.append(word)
    return (" ".join(clean))


def normalize_slang_words(text):
    slang_word_dict = json.loads(open('slang_word_dict.txt', 'r').read())
    words = text.split()
    for index in range(len(words)):
        for key, value in slang_word_dict.items():
            for v in value:
                if words[index] == v:
                    words[index] = key
                else:
                    continue 
    content = list()
    content.append(words)

    return " ".join(content)

def preprocessing_tweets_second(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    clean   = stemmer.stem(text) 
    clean = re.sub("#", "", clean)
    clean = stopword(clean.split()) #Stopword
    clean
#     clean = normalize_slang_words(clean)
    return clean

tweets_with_entity['clean'] = tweets_with_entity['text'].apply(preprocessing_tweets_second)


In [17]:
# tweets_with_entity.head(10)
export_csv = tweets_with_entity.to_csv (r'.\tweets_with_entity.csv', encoding="mbcs", header=True) #Don't forget to add '.csv' at the end of the path

clustering tweets

In [18]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
#     print(filtered_tokens)
    return filtered_tokens

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer


#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=1, max_features=200000,
                                 min_df = 0.2, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_with_entity['clean'].tolist()) #fit the vectorizer to synopses

Wall time: 91.8 ms


http://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/

In [23]:
print(tfidf_matrix.shape)

# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_matrix[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

(1, 6)


,tfidf
barat,0.408248
barat tuh,0.408248
budaya,0.408248
budaya barat,0.408248
budaya barat tuh,0.408248
tuh,0.408248


In [24]:
def cosine_distance(a, b, weigth = []):
    if len(weigth) == 0:
        weigth = np.array([1] * len(a))
    c_sim = np.sum(a * b * weigth) / (np.sqrt(sum(a ** 2)) * np.sqrt(sum(b ** 2)))
    return 1 - c_sim

import KMedoids



ModuleNotFoundError: No module named 'KMedoids'

In [ ]:
asddf

In [25]:
def silhouette(features, cluster_id):
    total = 0
    n = len(np.unique(cluster_id))
    for i, feat in enumerate(features): k.minute, 0)
        inner = 0
        outer = np.inf
        for j in range(n):
            dist = euclidean_distances(feat,features [cluster_id == j])[0]
        if j == cluster_id[i]:
            inner += dist.mean()
        else
            outer = min(outer, dist.mean())
        total += ((outer - inner) / max(outer,inner) if max(outer, inner) != 0 else 0)
    return total / len(features)

SyntaxError: invalid syntax (<ipython-input-25-fcfef3eb3a8b>, line 4)

In [250]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000, min_df=0.2 )
# X = vectorizer.fit_transform(tweets_with_entity['clean'].tolist())
# # print(vectorizer.get_feature_names())
# print(X.shape)

In [240]:
# print(tokenize_only(tweets['text'].tolist()))
print(tfidf_matrix.get_feature_names())

AttributeError: get_feature_names not found

In [20]:
# Clustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# documents = indonesia['clean'].tolist()
# vectorizer = TfidfVectorizer(stop_words=stopwords)
# X = vectorizer.fit_transform(documents)

true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(tfidf_matrix)


NameError: name 'tfidf_matrix' is not defined

In [241]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 aamiin hbdjokowi
 aamiin robbal
 aamiin
 aamiin moga lancar
 aamiin yra
 aamiin robbal alamin
 aamiin moga
 aaaaaamiiinnnn
Cluster 1:
 aamiin
 aamiin yra
 aamiin robbal alamin
 aamiin robbal
 aamiin moga lancar
 aamiin moga
 aamiin hbdjokowi
 aaaaaamiiinnnn
Cluster 2:
 aamiin robbal
 aamiin robbal alamin
 aamiin yra
 aaaaaamiiinnnn
 aamiin
 aamiin moga lancar
 aamiin hbdjokowi
 aamiin moga
Cluster 3:
 aaaaaamiiinnnn
 aamiin
 aamiin hbdjokowi
 aamiin moga lancar
 aamiin robbal
 aamiin moga
 aamiin yra
 aamiin robbal alamin
Cluster 4:
 aamiin moga
 aamiin
 aaaaaamiiinnnn
 aamiin moga lancar
 aamiin hbdjokowi
 aamiin robbal
 aamiin yra
 aamiin robbal alamin


print("\n")
print("Prediction")

Y = vectorizer.transform(["you are beautiful."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["jokowi terbaik."])
prediction = model.predict(Y)
print(prediction)

In [ ]:
#Kmedoids

cluster importance

representative words / sentences

trending topics

KMEDOIDS
https://github.com/letiantian/kmedoids

In [ ]:
# clearing and tokenizing
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

# Calculating bag of words
word_set = set(l_A).union(set(l_B)).union(set(l_C))

word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
word_dict_C[word] += 1

In [ ]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf
  
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

In [ ]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf
    
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])

In [ ]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf
    
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)

kmeans = KMeans(n_clusters=2).fit(tfidf)

KMEDOIDS

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

import kmedoids

# 3 points in dataset
data = np.array([[1,1], 
                [2,2], 
                [10,10]])

# distance matrix
D = pairwise_distances(data, metric='euclidean')

# split into 2 clusters
M, C = kmedoids.kMedoids(D, 2)

print('medoids:')
for point_idx in M:
    print( data[point_idx] )

print('')
print('clustering result:')
for label in C:
    for point_idx in C[label]:
        print('label {0}:　{1}'.format(label, data[point_idx]))

FILTRASI BERITA BERDASARKAN TRENDING ISSUE

In [267]:
# https://gokhanatil.com/2017/11/python-for-data-science-importing-xml-to-pandas-dataframe.html
import xml.etree.ElementTree as et 
import os
directory = "./ScrapeNews/berita/"
df = pd.DataFrame(columns=['judul','id','tanggal','kata_kunci','ísi'])

for berita in os.listdir(directory):
    filename = directory+berita
    xtree = et.parse(filename).getroot()
    judul = xtree.find('judul').text
    ids = xtree.find('id').text
    tanggal = xtree.find('tanggal').text
    kata_kunci = xtree.find('kata_kunci').text
    isi = xtree.find('isi').text
    df = df.append({'judul':judul,'id':ids,'tanggal':tanggal,'kata_kunci':kata_kunci,'ísi':isi},ignore_index=True)
#     df2 = pd.DataFrame([judul,id,tanggal,kata_kunci,isi], columns=['judul','id','tanggal','kata_kunci','ísi'])
#     df = df.append(df2)

pd.set_option('max_colwidth', 30)
# df

In [268]:
df.head(10)

,judul,id,tanggal,kata_kunci,ísi
0,Kronologi Kasus Korupsi Pl...,POL_POL_1,18/06/2019,"michel,platini,uefa,piala,...","Jakarta, CNN Indonesia -- ..."
1,Persib Imbang 1-1 dengan T...,POL_POL_2,18/06/2019,"persib,bandung,ps,tira,per...","Jakarta, CNN Indonesia -- ..."
2,Jadwal Copa America 2019: ...,POL_POL_3,18/06/2019,"copa,america,copa,america,...","Jakarta, CNN Indonesia -- ..."
3,Kasus Korupsi Platini Sere...,POL_POL_4,18/06/2019,"michel,platini,uefa,fifa,n...","Jakarta, CNN Indonesia -- ..."
4,ICW: Setya Novanto Jadi Ke...,POL_PP_02_025,17/05/16,,\n\t Terpilihnya Setya Nov...
5,Penutupan Kongres PDIP Bah...,POL_PP_02_1089,11/4/15,"RapimnasPDIP,Pilkada2015,...",\n Penutupan Kongres IV Pa...
6,Disambut Meriah di Rakerna...,POL_PP_02_1090,10/01/16,"RapimnasPDIP,Ahok","\n Gubernur DKI Jakarta, B..."
7,Silaturahmi Nasional Parta...,POL_PP_02_1091,02/11/15,"SilatnasGolkar,PecahGolka...",\n Wakil Sekretaris Jender...
8,SBY Apresiasi Jokowi yang ...,POL_PP_02_1092,12/05/15,"KongresDemokrat,Jokowi,SB...",\n Kongres IV Partai Demok...
9,"Prabowo Batal Hadir, Petin...",POL_PP_02_1093,12/05/15,"KongresDemokrat,KoalisiMe...",\nKetua Umum Gerindra Prab...


In [269]:
#Preprocessing berita
# factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()
# stopwords.extend(extender)


def preprocessing_news(text):
    clean = html.unescape(text) #Handling Unicode HTML
    clean = re.sub("[0-9]", "", clean) #RemoveNumber
    clean = " ".join(re.findall("[#a-zA-Z]{3,}", clean)) #Remove Puntc
    clean = clean.lower()
    content = list()
    content.append(clean)
    return " ".join(content)

# def preprocessing_news(text):
#     clean = html.unescape(text) #Handling Unicode HTML
#     clean = " ".join(re.findall("[#a-zA-Z]{3,}", clean)) #Remove Puntc
#     clean = text.lower() #lowercase # permrosesan NER sangat dipengaruhi huruf kapital
#     clean = stopword(clean) #Stopword
    return clean
#     content = list()
#     content.append(clean)
#     return " ".join(content)

df['clean'] = df['ísi'].apply(preprocessing_news)
df.head(10)

,judul,id,tanggal,kata_kunci,ísi,clean
0,Kronologi Kasus Korupsi Pl...,POL_POL_1,18/06/2019,"michel,platini,uefa,piala,...","Jakarta, CNN Indonesia -- ...",jakarta cnn indonesia pena...
1,Persib Imbang 1-1 dengan T...,POL_POL_2,18/06/2019,"persib,bandung,ps,tira,per...","Jakarta, CNN Indonesia -- ...",jakarta cnn indonesia pers...
2,Jadwal Copa America 2019: ...,POL_POL_3,18/06/2019,"copa,america,copa,america,...","Jakarta, CNN Indonesia -- ...",jakarta cnn indonesia timn...
3,Kasus Korupsi Platini Sere...,POL_POL_4,18/06/2019,"michel,platini,uefa,fifa,n...","Jakarta, CNN Indonesia -- ...",jakarta cnn indonesia nama...
4,ICW: Setya Novanto Jadi Ke...,POL_PP_02_025,17/05/16,,\n\t Terpilihnya Setya Nov...,terpilihnya setya novanto ...
5,Penutupan Kongres PDIP Bah...,POL_PP_02_1089,11/4/15,"RapimnasPDIP,Pilkada2015,...",\n Penutupan Kongres IV Pa...,penutupan kongres partai d...
6,Disambut Meriah di Rakerna...,POL_PP_02_1090,10/01/16,"RapimnasPDIP,Ahok","\n Gubernur DKI Jakarta, B...",gubernur dki jakarta basuk...
7,Silaturahmi Nasional Parta...,POL_PP_02_1091,02/11/15,"SilatnasGolkar,PecahGolka...",\n Wakil Sekretaris Jender...,wakil sekretaris jenderal ...
8,SBY Apresiasi Jokowi yang ...,POL_PP_02_1092,12/05/15,"KongresDemokrat,Jokowi,SB...",\n Kongres IV Partai Demok...,kongres partai demokrat ho...
9,"Prabowo Batal Hadir, Petin...",POL_PP_02_1093,12/05/15,"KongresDemokrat,KoalisiMe...",\nKetua Umum Gerindra Prab...,ketua umum gerindra prabow...


In [79]:
export_csv = df.to_csv (r'.\berita.csv', encoding="mbcs", header=True) #Don't forget to add '.csv' at the end of the path

def news_filtering(news_collection, terms):
    trending_news = list()
    for news in news_collection:
        result = False
        for sent in news:
            result = any (term in terms for word in sent)
            if result:
                trending_news.append(news)
                continue

PEMBOBOTAN KALIMAT BERITA

In [283]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
#     print(filtered_tokens)
    return filtered_tokens

In [284]:
# W1 menghitung Term Frequency
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df = 0.2, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_with_entity['clean'].tolist()) #fit the vectorizer to synopses



In [ ]:
# W2 TF-IDF
def frequency_scores(self, article_text):
    """ Individual (stemmed) word weights are then calculated for each
        word in the given article. Sentences are scored as the sum of their TF-IDF word frequencies.
    """

    # Add our document into the model so we can retrieve scores
    response = self._tfidf.transform([article_text])
    feature_names = self._tfidf.get_feature_names() # these are just stemmed words

    word_prob = {}  # TF-IDF individual word probabilities
    for col in response.nonzero()[1]:
        word_prob[feature_names[col]] = response[0, col]
    if DEBUG:
        print(word_prob)

    sent_scores = []
    for sentence in self.split_into_sentences(article_text):
        score = 0
        sent_tokens = self.tokenize_and_stem(sentence)
        for token in (t for t in sent_tokens if t in word_prob):
            score += word_prob[token]

        # Normalize score by length of sentence, since we later factor in sentence length as a feature
        sent_scores.append(score / len(sent_tokens))

    return sent_scores


In [ ]:
# W3 Posisi Kalimat
def position_score(self, i, size):
        """ Yields a value between (0,1), corresponding to sentence's position in the article.
            Assuming that sentences at the very beginning and ends of the article have a higher weight. 
            Values borrowed from https://github.com/xiaoxu193/PyTeaser
        """

        relative_position = i / size
        if 0 < relative_position <= 0.1:
            return 0.17
        elif 0.1 < relative_position <= 0.2:
            return 0.23
        elif 0.2 < relative_position <= 0.3:
            return 0.14
        elif 0.3 < relative_position <= 0.4:
            return 0.08
        elif 0.4 < relative_position <= 0.5:
            return 0.05
        elif 0.5 < relative_position <= 0.6:
            return 0.04
        elif 0.6 < relative_position <= 0.7:
            return 0.06
        elif 0.7 < relative_position <= 0.8:
            return 0.04
        elif 0.8 < relative_position <= 0.9:
            return 0.04
        elif 0.9 < relative_position <= 1.0:
            return 0.15
        else:
            return 0


In [ ]:
# W4 Kemiripan dengan Judul
def headline_score(self, headline, sentence):
    """ Gives sentence a score between (0,1) based on percentage of words common to the headline. """
    title_stems = [stemmer.stem(w) for w in headline if w not in stop_words]
    sentence_stems = [stemmer.stem(w) for w in sentence if w not in stop_words]
    count = 0.0
    for word in sentence_stems:
        if word in title_stems:
            count += 1.0
    score = count / len(title_stems)
    return score

In [ ]:
# W5 Kemiripan dengan Trending Issue
def simmilarity_trending_issue_score(self, headline, sentence):
    """ Gives sentence a score between (0,1) based on percentage of words common to the headline. """
    title_stems = [stemmer.stem(w) for w in headline if w not in stop_words]
    sentence_stems = [stemmer.stem(w) for w in sentence if w not in stop_words]
    count = 0.0
    for word in sentence_stems:
        if word in title_stems:
            count += 1.0
    score = count / len(title_stems)
    return score

In [ ]:
# W6 mengandung entitas bernama
def entity_score(self, headline, sentence):
    """ Gives sentence a score between (0,1) based on percentage of words common to the headline. """
    title_stems = [stemmer.stem(w) for w in headline if w not in stop_words]
    sentence_stems = [stemmer.stem(w) for w in sentence if w not in stop_words]
    count = 0.0
    for word in sentence_stems:
        if word in title_stems:
            count += 1.0
    score = count / len(title_stems)
    return score

In [ ]:
# W7 Panjang kalimat
def length_score(self, sentence):
        """ Gives sentence score between (0,1) based on how close sentence's length is to the ideal length."""
        len_diff = math.fabs(ideal_sent_length - len(sentence))
        return len_diff / ideal_sent_length


In [ ]:
# WTot 
def score(article):
        """ Assigns each sentence in the document a score based on the sum of features values.
            Based on 4 features: relevance to headline, length, sentence position, and TF*IDF frequency.
        """

        headline = article[0]
        sentences = self.split_into_sentences(article[1])
        frequency_scores = self.frequency_scores(article[1])

        for i, s in enumerate(sentences):
            headline_score = self.headline_score(headline, s) * 1.5
            length_score = self.length_score(self.split_into_words(s)) * 1.0
            position_score = self.position_score(float(i+1), len(sentences)) * 1.0
            frequency_score = frequency_scores[i] * 4
            score = (headline_score + frequency_score + length_score + position_score) / 4.0
            self._scores[s] = score

PENYUSUNAN RINGKASAN BERITA

In [ ]:
# mengurutkan berat berita
# menentukan jumlah kalimat hasil ringkasan
def generate_summaries(self):
    """ If article is shorter than the desired summary, just return the original articles."""

    # Rare edge case (when total num sentences across all articles is smaller than desired summary length)
    total_num_sentences = 0
    for article in self._articles:
        total_num_sentences += len(self.split_into_sentences(article[1]))

    if total_num_sentences <= SUMMARY_LENGTH:
        return [x[1] for x in self._articles]

    self.build_TFIDF_model()  # only needs to be done once

    self._scores = Counter()
    for article in self._articles:
        self.score(article)

    highest_scoring = self._scores.most_common(SUMMARY_LENGTH)
    if DEBUG:
        print(highest_scoring)

    print("## Headlines: ")
    for article in self._articles:
        print("- " + article[0])

    # Appends highest scoring "representative" sentences, returns as a single summary paragraph.
    return ' '.join([sent[0] for sent in highest_scoring])


In [ ]:
for i in range(N_CLUSTER):
    cluster_tf, cluster_terms =
    get_tf(non_trivial[cluster_id == i].tolist(),get_terms = True)
    cluster_tfidf = get_tfidf(non_trivial[cluster_id == i].tolist(),normalize = True)
    cluster_wf = np.array(get_wf(non_trivial[cluster_id == i].tolist()).values())
    selected = np.argwhere(np.logical_and(np.logical_and ((cluster_tfidf.max(axis = 0) > TFIDF), cluster_wf > WF), cluster_tf.max(axis = 0) > TF)).T[0]
    keywords.extend([cluster_terms[idx] for idx in selected])self.result_dict['result'][i]['selected_terms']= [cluster_terms[idx] for idx in selected]
    if len(self.result_dict['result'][i]['selected_terms']) > t_max:
        t_max = len(self.result_dict['result'][i]['selected_terms'])
        best_cluster = i
    self.result_dict['result'][i]['members'] = cluster_tf.shape[0]
self.result_dict['result'][i]['cluster_terms'] = cluster_terms
max_score = 0
for i in range(N_CLUSTER):
    term_clust = []
    for trm in non_trivial[cluster_id == i]:
        term_clust.extend(trm)
    score = 0
    for keyword in keywords:
        cou = term_clust.count(keyword)
        if cou > CI_THRES:
            score += np.log(term_clust.count(keyword) + 1)
        if max_score < score:
            max_score = score
            best_cluster = i
            self.result_dict['result'][i]['score'] = core
self.selected_terms_final = self.result_dict['result'][best_cluster]['select ed_terms']                                                                

UJI COBA